<a href="https://colab.research.google.com/github/Guthikonda-Akshaya/GanForge/blob/assignment-3/a3q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔍 Image Similarity Search using PyTorch + Annoy

This notebook demonstrates how to build an image similarity search engine using a pre-trained ResNet model for feature extraction and Spotify's Annoy library for fast nearest-neighbor search.

In [ ]:
# ✅ Install Annoy library
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553320 sha256=59847c1193c185e6274a13eb4de1e53bf61b544dd8fe8835f021cd0fbd538397
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [ ]:
# ✅ Import Libraries
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from PIL import Image
from annoy import AnnoyIndex
import os
from google.colab import files

In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload my_images.zip

Saving my_images.zip to my_images.zip


In [ ]:
import zipfile
with zipfile.ZipFile("my_images.zip", 'r') as zip_ref:
    zip_ref.extractall("data/images")

In [ ]:
# ✅ Load Pre-trained ResNet18 and remove final layer
resnet = models.resnet18(pretrained=True)
feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
feature_extractor.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 157MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [ ]:
# ✅ Image transformation for ResNet input
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### 🔍 Query with a test image

In [ ]:
# ✅ Load Dataset
dataset = datasets.ImageFolder('data/images', transform=transform)
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=False)

# ✅ Extract Embeddings
embeddings = []
with torch.no_grad():
    for batch, _ in loader:
        features = feature_extractor(batch).squeeze(-1).squeeze(-1)
        embeddings.extend(features.cpu().numpy())

# ✅ Build Annoy Index
vec_dim = len(embeddings[0])
annoy_index = AnnoyIndex(vec_dim, 'angular')
for i, emb in enumerate(embeddings):
    annoy_index.add_item(i, emb)
annoy_index.build(10)


True

In [ ]:
# ✅ Upload a query image
query_img = files.upload()
query_img_path = list(query_img.keys())[0]

# Preprocess query image
query_tensor = transform(Image.open(query_img_path).convert('RGB')).unsqueeze(0)
with torch.no_grad():
    q_vec = feature_extractor(query_tensor).squeeze().numpy()

# Retrieve top 5 similar images
idxs, dists = annoy_index.get_nns_by_vector(q_vec, 5, include_distances=True)

# Display results
print("\nTop 5 similar images:")
for i, d in zip(idxs, dists):
    img_path = dataset.imgs[i][0]
    print(f"{img_path} (distance: {d:.4f})")

Saving query_image.jpg to query_image (2).jpg

Top 5 similar images:
data/images/class_x/img_1.jpg (distance: 0.1297)
data/images/class_x/img_7.jpg (distance: 0.2835)
data/images/class_x/img_5.jpg (distance: 0.4961)
data/images/class_x/img_10.jpg (distance: 0.5526)
data/images/class_x/img_9.jpg (distance: 0.6033)
